In [7]:
import os
import sys 
import numpy as np 
import tensorflow as tf 
import time 
batch_size =50 

def load_model(session, save_path):
    """ Loads a saved TF model from a file.Returns:The inputs placehoder """
    print("Loading model from file '%s'..." % (save_path))
    meta_file = save_path + ".meta"
    if not os.path.exists(meta_file):
        print("ERROR: Expected .meta file '%s', but could not find it." % \
        (meta_file))
        sys.exit(1) 
    saver = tf.train.import_meta_graph(meta_file)
    save_path = os.path.join("./", save_path) 
    saver.restore(session, save_path)  
    return extract_validation_handles(session)
    
def load_validation_data( model,filename):
    '''load the testing dataset with the process and the name of testing file''' 
    global batch_size 
    """ Loads the validation data,  Returns: A tuple of the loaded validation data and validation labels. """
    print("Loading validation data...")
    folder = 'sub_features/'+ filename  
    if model ==1: # SP process
        testX =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testX_pred.npy' ) 
        testY =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testY_pred.npy'  ) 
        Energy =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/Energy_pred.npy' )
    elif model==2: # NS process
        testX =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testX_no.npy' ) 
        testY =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testY_no.npy' ) 
        Energy =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/Energy_no.npy'  ) 
    batch_size =np.shape(testX)[0]  
    return (testX,testY,Energy)  

def validate_model(session, val_data, x,y, e, phase_train,prob ): 
    """ Validates the model stored in a session.Returns:The overall validation accuracy for the model. """
    global batch_size
    print("Validating model...") 
    predict_op = tf.argmax(prob, 1)
    correct = tf.equal(predict_op,tf.argmax(y, 1))
    acc= tf.reduce_mean(tf.cast(correct, tf.float32))  
    val_x, val_y, val_e = val_data
    test_correct = session.run(correct,feed_dict={x: val_x, e:val_e,y :val_y , phase_train : True  })
    test_acc = session.run(acc,feed_dict={x: val_x, e:val_e,y :val_y , phase_train : True  })
    return ( test_acc), test_correct  

def extract_validation_handles(session):
    """ Extracts the input and predict_op handles that we use for validation.
    Args:
      session: The session with the loaded graph.
    Returns:
      The inputs placeholder  """ 
    valid_nodes = tf.get_collection_ref("validation_nodes") 
    x = valid_nodes[0] 
    y = valid_nodes[1]
    e = valid_nodes[2]  
    phase_train =valid_nodes[3]
    prob = valid_nodes[4]
    global batch_size  
    return (x,y,e, phase_train,prob)

def each_perform(correct_results,eval_labels ):
    label_y=np.argmax(eval_labels,1) 
    
    acc = []
    zero = np.where(label_y==0) 
    correctl=[correct_results[i] for i in zero]
    accuracy_zero= np.mean(correctl,dtype=np.float32)
    acc.append(accuracy_zero)

    one = np.where(label_y==1)
    correctg=[correct_results[i] for i in one] 
    accuracy_one= np.mean(correctg,dtype=np.float32)
    acc.append(accuracy_one)

    two = np.where(label_y==2)
    correctt=[correct_results[i] for i in two] 
    accuracy_two= np.mean(correctt,dtype=np.float32)
    acc.append(accuracy_two) 
    return acc, correctl,correctg,correctt


def reset():
    tf.reset_default_graph()

  
  
model = 1 # model = 1: SP process; model = 2: NS process
folder ='fully_best/modelA_ratio100_set01'#'modelA_fully3'##''modelA_ratio100_set3'#'modelA_ratio80_set4'#'modelA_ratio50_set2'#'modelA_ratio30_set2'#'modelA_fully3'#'modelA_ratio24_set05'#'modelA_small04' # 'modelA_fully3' #,'save2','save3','save4','save5','save6','save7','save8','save9','save10','save11','save12','save13','save14','save15' ]
testname = 'total'
with tf.Session() as session:  
    best_model = 'E:/05_research/04_deep_learning_model/Save_parameters/modelA/'+folder# os.path.join(save_path,model_name)
    x,y,e,phase_train,prob = load_model(session, best_model)
    val_data = load_validation_data(model, testname) # the predicted mode
    accuracy ,correct_results= validate_model(session, val_data, x,y,e,phase_train,prob)
    print ("Overall validation accuracy is %f \n" %(accuracy))
    acc,correct_l,correct_g,correct_t= each_perform(correct_results,val_data[1] ) 
    #reset() 
    session.close()
each_perform(correct_results,val_data[1] )     
print ('LT:', 100*np.mean( correct_l))
print ('GT:', 100*np.mean( correct_g))
print ('TP:', 100*np.mean( correct_t))
 
    

Loading model from file 'E:/05_research/04_deep_learning_model/Save_parameters/modelA/fully_best/modelA_ratio100_set01'...
INFO:tensorflow:Restoring parameters from E:/05_research/04_deep_learning_model/Save_parameters/modelA/fully_best/modelA_ratio100_set01
Loading validation data...
Validating model...
Overall validation accuracy is 0.888273 

LT: 82.8042328042328
GT: 85.30805687203792
TP: 96.96969696969697


In [8]:
id_false =  np.nonzero(correct_results==False)  
id_false

(array([  18,   23,   25,   31,   33,   38,   64,   69,   70,   74,   77,
          78,   82,   83,   85,   88,   90,  101,  104,  118,  143,  156,
         162,  200,  244,  250,  280,  286,  315,  374,  425,  490,  547,
         572,  601,  619,  620,  623,  634,  638,  640,  641,  642,  644,
         647,  650,  660,  686,  712,  734,  736,  742,  743,  744,  787,
         813,  814,  815,  816,  822,  824,  826,  827,  829,  830,  831,
         834,  836,  838,  839,  840,  841,  842,  843,  845,  848,  849,
         852,  853,  855,  857,  860,  870,  874,  882,  884,  888,  905,
         906,  914,  920,  924,  925,  928,  930,  931,  932,  933,  934,
         953,  975,  987,  998,  999, 1022, 1023, 1027, 1043, 1051, 1061,
        1090, 1115, 1134, 1152, 1153, 1155, 1176, 1179, 1180, 1181, 1182,
        1184, 1185, 1188, 1195, 1196, 1197, 1218, 1224, 1230, 1232, 1249,
        1250, 1251, 1252, 1253, 1256, 1257, 1258, 1259, 1260], dtype=int64),)

In [9]:
id_f  =[  18,   23,   25,   31,   33,   38,   64,   69,   70,   74,   77,
          78,   82,   83,   85,   88,   90,  101,  104,  118,  143,  156,
         162,  200,  244,  250,  280,  286,  315,  374,  425,  490,  547,
         572,  601,  619,  620,  623,  634,  638,  640,  641,  642,  644,
         647,  650,  660,  686,  712,  734,  736,  742,  743,  744,  787,
         813,  814,  815,  816,  822,  824,  826,  827,  829,  830,  831,
         834,  836,  838,  839,  840,  841,  842,  843,  845,  848,  849,
         852,  853,  855,  857,  860,  870,  874,  882,  884,  888,  905,
         906,  914,  920,  924,  925,  928,  930,  931,  932,  933,  934,
         953,  975,  987,  998,  999, 1022, 1023, 1027, 1043, 1051, 1061,
        1090, 1115, 1134, 1152, 1153, 1155, 1176, 1179, 1180, 1181, 1182,
        1184, 1185, 1188, 1195, 1196, 1197, 1218, 1224, 1230, 1232, 1249,
        1250, 1251, 1252, 1253, 1256, 1257, 1258, 1259, 1260]

In [10]:
id1 = [i for i in id_f if i < 169]
id2 = [i for i in id_f if i >= 169 and i < (169+404)]
id3 = [i for i in id_f if i >= (169+404) and i < (169+404+378)]
id4 = [i for i in id_f if i >= (169+404+378) and i < (169+404+378+107)]
id5 = [i for i in id_f if i >= (169+404+378+107) and i < (169+404+378+107+58)]
id6 = [i for i in id_f if i >= (169+404+378+107+58) and i < 1262]


In [11]:
acc1 = (1-len(id1)/169)*100 ; print ( acc1)
acc2 = (1-len(id2)/404)*100 ; print ( acc2)
acc3 = (1-len(id3)/378)*100 ; print ( acc3)
acc4 = (1-len(id4)/107)*100 ; print ( acc4)
acc5 = (1-len(id5)/58)*100 ; print ( acc5)
acc6 = (1-len(id6)/146)*100 ; print ( acc6)


86.3905325443787
97.27722772277228
82.8042328042328
90.65420560747664
94.82758620689656
80.13698630136986
